In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Concatenate, Add, Multiply,Average, Subtract
from tensorflow.keras import Model
import traffic_data_generator
import traffic_util

Fix the random seeds in numpy, tensorlfow

In [3]:
import random as rn

rand_seed=116
rn.seed(rand_seed)
np.random.seed(rand_seed)
tf.random.set_seed(rand_seed)

In [4]:
example_name="traffic"
data_dir=traffic_data_generator.set_up(example_name)

In [5]:
delta_gen=0.05
n_traj=25000
n_samples=100000

In [6]:
#===========================
#     Output directories 
#===========================
experiment_name="_no_share"+ \
     "_"+str(n_samples)+"_Delta_"+str(delta_gen)


model_dir=example_name+"/"+experiment_name
plot_dir=model_dir+"/Plots"
checkpoint_dir=model_dir+"/check_points"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

In [8]:
# inits=['Orthogonal', 'glorot_normal','he_normal' ,'he_uniform', 'glorot_uniform']
n_starts=6
inits=['glorot_normal','he_normal' ,'he_uniform']
rand_seed_shift=0
n_epochs=2000
# combinations=np.array([[20,.5],[4,2.5]]) old combinations
combinations=np.array([[64,.25],[40,.25],[24,.25],[16,.25],[8,.25],[32,.5],[20,.5],[12,.5],[8,.5],[4,.5],
                       [16,1],[10,1],[6,1],[4,1],[2,1],[8,2],[5,2],[3,2],[2,2]])
n_new_combo=len(combinations)
n_starts=4
validation_split=.2
np.savetxt(data_dir+'/combinations'+ str(len(combinations)) +'.txt', combinations)
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=200, verbose=1, mode='min',
    baseline=None, restore_best_weights=True
)
ith_run=12
print('ith_run',ith_run)


ith_run 8


In [13]:
## below is for i_combo=0 and ith_run=0
# for i_start=1  we use the hard_sigmoid function as our second to last activation funciton (for intial guess)
# for i_start=2 we multiply the input of hard_sigmoid by 10 to get it close to heavyiside (didn't work)
# for i_start=3 we try to use L-1 norm
# for i_start=4 we use NN data and revert to tanh and L-2 norm
# for i_start=5 we use polyfit data (degree 5)
## below is for ith run=1 i_combo=0
# for i_start=1 we run with no lr deca
## below is for ith_run=2 (uniform data generation) with 20 sec traj len
##below is for ith_run=3 uniform data gen with updated interaction rules and 100 sec traj len
#for i_start=1 we use abs val weight
##for ith_run=5 
# i_start=0 is "normal"
# I_start=1 we use sigmoid
# I_start=2 we use polyfit
# I_start=3 we use abs weighted
for i_combo in range(len(combinations)):
    for i_start in (0,2,3):
        opt_model_path=checkpoint_dir+"/opt_model_first_try" +str(i_start)+ '_' +str(i_combo) + '_' +str(ith_run) +".h5"
        if not os.path.exists(opt_model_path):
#         if True:
            print([i_combo,i_start,ith_run])
            i_hist_MZ=combinations[i_combo,0]
            Delta_t=combinations[i_combo,1]
            len_hist_MZ=int(i_hist_MZ)
            if i_start==2:
                x=np.load(f'{data_dir}/poly_deg=5_x_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
                y_temp=np.load(f'{data_dir}/poly_deg=5_y_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
            elif i_start==3:
                x=np.load(f'{data_dir}/abs_weight_smoothed_x_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
                y_temp=np.load(f'{data_dir}/abs_weight_smoothed_y_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
            else:
                x=np.load(f'{data_dir}/x_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
                y_temp=np.load(f'{data_dir}/y_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
            exec('y_shift'+str(i_combo)+'_'+str(i_start)+'=traffic_util.min_max_scale()')
            exec('y=y_shift'+str(i_combo)+'_'+str(i_start)+'.fit_transform((y_temp-x[:,0,:]))')
            tf.random.set_seed(rand_seed_shift+i_start+i_combo*len(combinations))
            if x.shape[0]==y.shape[0]:
                n_data=x.shape[0]
            else:
                break
            x=x.reshape([n_data,-1])
            x_test=x[:int(validation_split*len(x))]
            x_train=x[int(validation_split*len(x)):]
            y_test=y[:int(validation_split*len(y))]
            y_train=y[int(validation_split*len(y)):]
            
            states = Input(shape=(x.shape[1],))
            x1=states
            x2=Dense(150, activation='relu',kernel_initializer=inits[i_start%(len(inits)-1)])(x1)
            x3=Dense(100, activation='relu',kernel_initializer=inits[i_start%(len(inits)-1)])(x2)
            if i_start==1:
                x4=Dense(70, activation='hard_sigmoid',kernel_initializer=inits[i_start%(len(inits)-1)])(x3*10)
            else:
                x4=Dense(70, activation='tanh',kernel_initializer=inits[i_start%(len(inits)-1)])(x3)
            x_output=Dense(y.shape[1],activation='linear')(x4)
            outputs=x_output
            model=Model(inputs=states, outputs=outputs)  
            learning_rate=0.001
            batch_size=30
            adam=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.998, epsilon=10**-8, decay=0.0, amsgrad=False)
            model.compile(optimizer=adam, loss='mean_squared_error')
            savebest= tf.keras.callbacks.ModelCheckpoint(
                                filepath=opt_model_path, monitor='val_loss', verbose=i_combo==0, save_best_only=True,
                                save_weights_only=False, mode='auto', save_freq='epoch'
                                )
            #     lr_decrease=bursting_util.ReduceLROnPlateau(
            #                         monitor='val_loss', factor=0.2, patience=10, verbose=1, mode='auto',
            #                         min_delta=0, cooldown=0, min_lr=10**-12,filepath=opt_model_path
            #                         )
            lr_decrease=tf.keras.callbacks.ReduceLROnPlateau(
                            monitor='val_loss', factor=0.5, patience=50, verbose=i_combo==0, mode='auto',
                            min_delta=0, cooldown=0, min_lr=10**-15
                            )
            hist = model.fit(x_train, y_train, 
                          batch_size=batch_size,
                          epochs=n_epochs, 
                          verbose=i_combo==0,
                          callbacks=[savebest,lr_decrease,early_stop],
                          validation_data=(x_test,y_test),
                        )
            val_loss_log=np.concatenate((hist.history['val_loss'],100*np.ones(n_epochs-len(hist.history['val_loss']))))
            loss_log=np.concatenate((hist.history['loss'],100*np.ones(n_epochs-len(hist.history['loss']))))
            np.save(checkpoint_dir+"/loss_opt_model_first_try" +str(i_start)+ '_' +str(i_combo) + '_' +str(ith_run) +".npy",loss_log)
            np.save(checkpoint_dir+"/val_loss_opt_model_first_try" +str(i_start)+ '_' +str(i_combo) + '_' +str(ith_run) +".npy",val_loss_log)
            

[0, 3, 8]
Train on 80000 samples, validate on 20000 samples
Epoch 1/200
79650/80000 [============================>.] - ETA: 0s - loss: 0.0077
Epoch 00001: val_loss improved from inf to 0.00584, saving model to traffic/_no_share_100000_Delta_0.05/check_points/opt_model_first_try3_0_8.h5
80000/80000 [==============================] - 10s 123us/sample - loss: 0.0077 - val_loss: 0.0058
Epoch 2/200
79770/80000 [============================>.] - ETA: 0s - loss: 0.0058
Epoch 00002: val_loss improved from 0.00584 to 0.00491, saving model to traffic/_no_share_100000_Delta_0.05/check_points/opt_model_first_try3_0_8.h5
80000/80000 [==============================] - 10s 120us/sample - loss: 0.0058 - val_loss: 0.0049
Epoch 3/200
79650/80000 [============================>.] - ETA: 0s - loss: 0.0051
Epoch 00003: val_loss improved from 0.00491 to 0.00447, saving model to traffic/_no_share_100000_Delta_0.05/check_points/opt_model_first_try3_0_8.h5
80000/80000 [==============================] - 10s 123u

Epoch 63/200
79560/80000 [============================>.] - ETA: 0s - loss: 0.0024
Epoch 00063: val_loss did not improve from 0.00212
80000/80000 [==============================] - 9s 108us/sample - loss: 0.0024 - val_loss: 0.0023
Epoch 64/200
79470/80000 [============================>.] - ETA: 0s - loss: 0.0024
Epoch 00064: val_loss did not improve from 0.00212
80000/80000 [==============================] - 9s 113us/sample - loss: 0.0024 - val_loss: 0.0022
Epoch 65/200
79950/80000 [============================>.] - ETA: 0s - loss: 0.0024
Epoch 00065: val_loss did not improve from 0.00212
80000/80000 [==============================] - 9s 110us/sample - loss: 0.0024 - val_loss: 0.0026
Epoch 66/200
79530/80000 [============================>.] - ETA: 0s - loss: 0.0024
Epoch 00066: val_loss did not improve from 0.00212
80000/80000 [==============================] - 9s 112us/sample - loss: 0.0024 - val_loss: 0.0023
Epoch 67/200
79890/80000 [============================>.] - ETA: 0s - loss: 

79500/80000 [============================>.] - ETA: 0s - loss: 0.0020
Epoch 00130: val_loss did not improve from 0.00179
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0020 - val_loss: 0.0018
Epoch 131/200
79410/80000 [============================>.] - ETA: 0s - loss: 0.0021
Epoch 00131: val_loss did not improve from 0.00179
80000/80000 [==============================] - 9s 106us/sample - loss: 0.0021 - val_loss: 0.0019
Epoch 132/200
79590/80000 [============================>.] - ETA: 0s - loss: 0.0021
Epoch 00132: val_loss did not improve from 0.00179
80000/80000 [==============================] - 9s 110us/sample - loss: 0.0021 - val_loss: 0.0023
Epoch 133/200
79710/80000 [============================>.] - ETA: 0s - loss: 0.0020
Epoch 00133: val_loss did not improve from 0.00179
80000/80000 [==============================] - 9s 118us/sample - loss: 0.0020 - val_loss: 0.0024
Epoch 134/200
79890/80000 [============================>.] - ETA: 0s - loss: 0.0020
Ep

Epoch 165/200
79950/80000 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00165: val_loss did not improve from 0.00177
80000/80000 [==============================] - 9s 108us/sample - loss: 0.0019 - val_loss: 0.0021
Epoch 166/200
79500/80000 [============================>.] - ETA: 0s - loss: 0.0020
Epoch 00166: val_loss did not improve from 0.00177
80000/80000 [==============================] - 9s 107us/sample - loss: 0.0020 - val_loss: 0.0024
Epoch 167/200
79680/80000 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00167: val_loss did not improve from 0.00177
80000/80000 [==============================] - 10s 123us/sample - loss: 0.0019 - val_loss: 0.0022
Epoch 168/200
79860/80000 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00168: val_loss did not improve from 0.00177
80000/80000 [==============================] - 10s 123us/sample - loss: 0.0019 - val_loss: 0.0018
Epoch 169/200
79920/80000 [============================>.] - ETA: 0s -

Epoch 199/200
79620/80000 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00199: val_loss did not improve from 0.00166
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0019 - val_loss: 0.0019
Epoch 200/200
79440/80000 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00200: val_loss did not improve from 0.00166
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0019 - val_loss: 0.0024
[1, 3, 8]
[2, 3, 8]
[3, 3, 8]
[4, 3, 8]
[5, 3, 8]
[6, 3, 8]
[7, 3, 8]
[8, 3, 8]
[9, 3, 8]
[10, 3, 8]
[11, 3, 8]
[12, 3, 8]
[13, 3, 8]
[14, 3, 8]
[15, 3, 8]
[16, 3, 8]
[17, 3, 8]
[18, 3, 8]
